In [42]:
import pandas as pd
import re
import pprint

In [43]:
data = pd.read_csv('reg-condition-active.csv', delimiter = ',')

data.head()

,courseno,title_long_en,degree_level,pre_en,year_end,open_status
0,100316,Special Education for Enhancing Potentials of ...,U,100112 and third year standing; or 100112 and ...,2599.0,1
1,206787,Theory of Statistics for Teachers,G,"206332 and one year courses in statistic, basi...",2599.0,1
2,703798,Independent Study,G,Students are required to submit independent st...,2599.0,1
3,1320,English and Science and Future Trends,U,001101 AND 001102 AND 001201 AND 001202; OR 00...,2599.0,1
4,204482,Simulation and Modelling,U,204203 or 204211 or 204214 or 204215 or 204216...,2599.0,1


In [44]:
def split_out_paren(s, token) :
	result = ""
	cur_str = ""
	in_paren = ""
	open = False
	for i in range(len(s)) :
		if s[i] == '(' and open :
			raise Exception("There is two nested loop !")
		if s[i] == '(' :
			open = True
			in_paren += s[i]
			result += cur_str + ' '
			cur_str = ""
		elif open and s[i] == ')' :
			open = False
			in_paren += s[i]
			token.append(in_paren)
			in_paren = ""
		elif open :
			in_paren += s[i]
		else :
			cur_str += s[i]
	result += cur_str
	result = re.sub(' +', ' ', result.strip())
	return result

def split_out_paren_test() :
	input = ["261261 (261215, 245874) 236548",
			 "(261215, 245874) 236548",
			 "(261215, 245874)",
			 "261254 (261215, 245874)",
			 "261254, 261215",
			 "261216(261215)262262"
			]
	output = ["261261 236548",
			  "236548",
			  "",
			  "261254",
			  "261254, 261215",
			  "261216 262262"
			 ]
	token = [["(261215, 245874)"],
			 ["(261215, 245874)"],
			 ["(261215, 245874)"],
			 ["(261215, 245874)"],
			 [],
			 ["(261215)"]
			]
	for i in range(len(input)) :
		cur_token = []
		assert(split_out_paren(input[i], cur_token) == output[i])
		assert(cur_token == token[i])

split_out_paren_test()

In [47]:
lookup = dict()

for index, row in enumerate(data) :
	paren = []
	condition = str(row[3]).lower()
	condition = split_out_paren(condition, paren)
	splited = re.split(';| and | or ', condition)
	splited = [(lambda s : s.strip())(s) for s in splited]
	splited = list(filter(None, splited))
	for s in splited :
		if s in lookup and lookup[s] != None and len(lookup[s]) < 5 :
			tmp = lookup[s]
			tmp.append(row[0])
			lookup[s] = tmp
		else :
			lookup[s] = [row[0]]
	for s in paren :
		if s in lookup and lookup[s] != None and len(lookup[s]) < 5:
			tmp = lookup[s]
			tmp.append(row[0])
			lookup[s] = tmp
		else :
			lookup[s] = [row[0]]

lookup_not_simple = dict((key, lookup[key]) for key in dict(lookup) if not str(key).isnumeric())

with open('lookup.txt', 'w') as output :
	pprint.pprint(lookup_not_simple, output)